In [31]:
import pandas as pd
import requests
import json
import os as os
import logging

In [26]:
pathforjsons = "C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response"

In [27]:
os.chdir(pathforjsons)

In [28]:

body={
   "email" : "dinesh@4basecare.com",
   "password" :"dinesh@123"
}
token = requests.post("https://www.v2.api.ecrf.4basecare.co.in/user/login", json=body).json()['payLoad']['authToken']
patient = 'IN-423-VBVD'
url = f"https://www.v2.api.ecrf.4basecare.co.in/integration/getExternalApiResponseByPatientId/{patient}"
headers = {'Authorization': f'Bearer {token}'}
rs = requests.get(url, headers = headers).json()
rs

{'success': True,
 'message': 'successfully fetched',
 'payLoad': {'patientInfo': {'patientId': 'IN-423-VBVD',
   'patientUUId': 'f1ba8628-618c-4001-b130-60139390ee80',
   'bmi': None,
   'preMenopausecycleType': None,
   'preMenopausecycleDuration': None,
   'age': 52,
   'gender': 'FEMALE',
   'menstruation': None,
   'occupation': None,
   'ethnicity': None,
   'height': 164.0,
   'weight': 58.0,
   'diet': 'Vegetarian',
   'maritalStatus': None,
   'numberOfChildren': 2,
   'parity': None},
  'medicalInfo': {'approximateDate': None,
   'ageAtDiagnosis': 50,
   'symptomsBeforeDiagnosis': 'Coughing ',
   'hereditaryFactor': 'SIGNIFICANT',
   'pastHistoryFactor': None,
   'hereditaryFactorSignificant': None,
   'synchronousOrMetachronous': None,
   'hereditaries': [{'patientRelation': 'Father',
     'cancerSiteName': None,
     'ageOfDiagnosis': None}],
   'caseSummaryInfos': [{'reportedDate': '2022-02-16',
     'source': None,
     'hospitalNameCSD': None,
     'hospitalLocationNameC

In [35]:

api_url = "https://www.v2.api.ecrf.4basecare.co.in/integration/getExternalApiResponseByPatientId/"
pat_df = pd.read_csv("C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/samples.tsv", sep='\t')
# List of patient IDs
patient_ids= pat_df['samples'].to_list()
open('data_export.log', 'a').close()
logging.basicConfig(filename='data_export.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
patient_ids[:5]

['IN-423-VKVG', 'IN-423-VCGB', 'IN-423-VI3Q', 'IN-423-VI3O', 'IN-423-VHWO']

In [41]:
def fetch_and_export_data(patient_id, token):
    url = f"{api_url}{patient_id}"
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse the JSON response
        patient_data = response.json()
        
        # Export the patient data as a JSON file
        output_file = f"{patient_id}_data.json"
        with open(output_file, 'w') as json_file:
            json.dump(patient_data, json_file, indent=2)
        logging.info(f"Data for {patient_id} exported successfully.")
    else:
        logging.error(f"Error fetching data for {patient_id}. Status code: {response.status_code}")

In [42]:
# Loop through each patient ID and fetch/export data
for patient_id in patient_ids:
    fetch_and_export_data(patient_id, token)

In [9]:
json_list=os.listdir(os.getcwd())
json_list=[i for i in json_list if i.endswith('.json')]

In [11]:
def json_cd(info,path, output_path):
    df = pd.read_json(path)
    df.drop(['success', 'message'], axis=1, inplace=True)
    df=df.T
    df.to_json
    df[info].to_json(output_path, orient='records', indent=2)
    print(f'Cancer details exported to {output_path} with indentations')

In [19]:
os.chdir('C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/')

In [20]:
for i in json_list:
    json_cd(i, f"C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/medical_info/{i}_MD.json")

Cancer details exported to C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/medical_info/IN-423-UI3A_data.json_MD.json with indentations
Cancer details exported to C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/medical_info/IN-423-UJHA_data.json_MD.json with indentations
Cancer details exported to C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/medical_info/IN-423-UL2H_data.json_MD.json with indentations
Cancer details exported to C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/medical_info/IN-423-VADB_data.json_MD.json with indentations
Cancer details exported to C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/medical_info/IN-423-VBUB_data.json_MD.json with indentations
Cancer details exported to C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/medical_info/IN-423-VC2B_data.json_MD.json with indentations
Cancer details exported to C:/Users/prksh/Downloads/

In [21]:
os.chdir("C:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/medical_info/")

In [22]:
cd_list = [file for file in os.listdir(os.getcwd()) if file.endswith('.json')]
cd_list

['IN-423-UI3A_data.json_MD.json',
 'IN-423-UJHA_data.json_MD.json',
 'IN-423-UL2H_data.json_MD.json',
 'IN-423-VADB_data.json_MD.json',
 'IN-423-VBUB_data.json_MD.json',
 'IN-423-VC2B_data.json_MD.json',
 'IN-423-VC2C_data.json_MD.json',
 'IN-423-VC2E_data.json_MD.json',
 'IN-423-VC2F_data.json_MD.json',
 'IN-423-VC2G_data.json_MD.json',
 'IN-423-VC2H_data.json_MD.json',
 'IN-423-VC5B_data.json_MD.json',
 'IN-423-VC5C_data.json_MD.json',
 'IN-423-VCCA_data.json_MD.json',
 'IN-423-VCCC_data.json_MD.json',
 'IN-423-VCDA_data.json_MD.json',
 'IN-423-VCGB_data.json_MD.json',
 'IN-423-VCGH_data.json_MD.json',
 'IN-423-VCGI_data.json_MD.json',
 'IN-423-VCGJ_data.json_MD.json',
 'IN-423-VCHB_data.json_MD.json',
 'IN-423-VCIA_data.json_MD.json',
 'IN-423-VCIB_data.json_MD.json',
 'IN-423-VCNA_data.json_MD.json',
 'IN-423-VCYC_data.json_MD.json',
 'IN-423-VD1G_data.json_MD.json',
 'IN-423-VD2A_data.json_MD.json',
 'IN-423-VD2C_data.json_MD.json',
 'IN-423-VD3D_data.json_MD.json',
 'IN-423-VD3E_

# Test

In [39]:
from pandas.io.json import json_normalize
# Read the JSON file
with open('c:/Users/prksh/Downloads/CLusteringTR/Clustering_Pilot2/json_response/IN-423-UI3A_data.json') as f:
    json_data = json.load(f)

# Remove the outer square brackets
#json_data = json_data[0][0]

# Use json_normalize to flatten the nested structure
df = pd.json_normalize(json_data, sep='_')
df

,success,message,payLoad_patientInfo_patientId,payLoad_patientInfo_patientUUId,payLoad_patientInfo_bmi,payLoad_patientInfo_preMenopausecycleType,payLoad_patientInfo_preMenopausecycleDuration,payLoad_patientInfo_age,payLoad_patientInfo_gender,payLoad_patientInfo_menstruation,...,payLoad_medicalInfo_isMedicalAllergies,payLoad_medicalInfo_medicalAllergiesSpecify,payLoad_medicalInfo_immunosupression,payLoad_medicalInfo_imunosupressionSpecify,payLoad_medicalInfo_otherFormsOfTobaccoUse,payLoad_medicalInfo_otherFormsOfTobaccoUseSpecify,payLoad_medicalInfo_cancerDetailsComments,payLoad_medicalInfo_dateOfDiagnosis,payLoad_cancerDetails,payLoad_events
0,True,successfully fetched,IN-423-UI3A,5b46aee0-5b3f-4ff1-86ff-8527736c2f6b,26.4,None,None,64,MALE,None,...,True,Penicillin Allergy,Unknown,None,Unknown,None,None,None,"[{'cancerSiteIdcCode': 'C34', 'cancerSite': 'L...","[{'eventType': 'DIAGNOSTIC_TEST', 'eventDate':..."


In [23]:
merged_df=[]
for i in cd_list:
    with open(i) as f:
        json_data = json.load(f)
    # Remove the outer square brackets
    if not json_data or not json_data[0]:
        print("JSON structure is empty or does not contain any fields.")
    else:
     json_data = json_data[0]
     df = pd.json_normalize(json_data, sep='_')
     df['Sample'] = i.replace('_data.json_MD.json', '')
     merged_df.append(df)
    

In [28]:
if not json_data or not json_data[0]:
    print("JSON structure is empty or does not contain any fields.")

JSON structure is empty or does not contain any fields.


In [24]:
pd.concat(merged_df).to_csv("C:/Users/prksh/Downloads/CLusteringTR/pilot2_details_md.csv")

In [65]:
merged_df

[     patientId                           patientUUId   bmi  \
 0  IN-423-UI3A  5b46aee0-5b3f-4ff1-86ff-8527736c2f6b  26.4   
 
   preMenopausecycleType preMenopausecycleDuration  age gender menstruation  \
 0                  None                      None   64   MALE         None   
 
   occupation ethnicity  height  weight        diet maritalStatus  \
 0       None      None   177.8    83.4  Vegetarian          None   
 
    numberOfChildren parity                         Sample  
 0                 2   None  IN-423-UI3A_data.json_PD.json  ,
      patientId                           patientUUId   bmi  \
 0  IN-423-UJHA  d50dcfb0-3d44-475d-ac1e-8299e3ffb8f1  18.9   
 
   preMenopausecycleType preMenopausecycleDuration  age  gender menstruation  \
 0                  None                      None   71  FEMALE         None   
 
   occupation ethnicity  height  weight        diet maritalStatus  \
 0       None      None   162.5    50.0  Vegetarian          None   
 
    numberOfChildre